# Introducción

La idea es scrapear hoteles y departamentos de una página de rentas venezolana (a pesar de que soy argentino). La idea surgió de leer [este proyecto](https://www.workana.com/job/scraping-a-pagina-web?ref=projects_1) en una página de trabajos freelance. Excepto que decidí cambiarla un poco.

# Extracción (scrapeo) de la información

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
base_url = "https://rentahouse.com.ve/"

In [3]:
# Consigue los links a cada publicacion dentro de cada pagina de una seccion (casas por defecto)
def get_links(base_url, page=1, casa=True):
    if casa:
        url = base_url + "casa_en_venta.html"
        payload = { 'page' : page, 'orderBy' : 'entryTimestamp desc', 'propertyTypeFromSlug' : 'Casa' }
    else:
        url = base_url + "apartamento_en_venta.html"
        payload = { 'page' : page, 'orderBy' : 'entryTimestamp desc', 'propertyTypeFromSlug' : 'Apartamento' }
    
    req = requests.get(url, params=payload)
    soup = BeautifulSoup(req.text, "html.parser")
    homes = soup.find_all("div", class_='property-list')

    res = []
    for home in homes:
        res.append(home.a['href'])
    
    return res

In [4]:
links = get_links(base_url)

In [5]:
# Extrae la informacion (sucia) de una publicacion
def get_info_from_house(url):
    req = requests.get(url)
    
    if req.status_code != 200:
        print("No se puede acceder a " + url)
        return
    
    soup = BeautifulSoup(req.text, "html.parser")
    
    try:
        info = soup.find('div', class_="propertyInfo")
        details = info.find('div', class_="row")
    except AttributeError as e:
        print("La pagina tiene algun error.")
        return
    
    data = {}

    data['Price'] = details.find('div', class_='price')['title']

    general_info = details.find('ul', class_="property-detailes-list").find_all('li')

    for item in general_info:
        spans = item.find_all('span')
        data[spans[0].text] = spans[1].text

    location = details.find_all('div', class_='DescripcionGeneral')[-1].find_all('li')

    for item in location:
        spans = item.find_all('span')
        data[spans[0].text] = spans[1].text

    return data

In [6]:
get_info_from_house(links[0])

{'Price': '$150,000 USD',
 'Codígo RAH:': 'VE 23-26807',
 'Tipo de Propiedad:': 'Casa',
 'Estilo:': 'Duplex',
 'Área Privada:': '225 m2\n\n',
 'Terreno:': '225 m2\n\n',
 'Estado Del Inmueble:': 'Usado',
 'Dormitorios:': '4',
 'Total Baños:': '5',
 'Baños Completos:': '4',
 'Medios Baños:': '1',
 'Tipo De Estacionamiento:': 'Cubierto',
 'Puestos De Estacionamiento:': '2',
 'Amoblado:': 'No',
 '✅Dormitorio De Servicio:': 'Si',
 'Calle:': 'Publica',
 'País: ': 'Venezuela',
 'Estado: ': 'Distrito Metropolitano',
 'Ciudad: ': 'Caracas',
 'Urbanización: ': 'La Boyera'}

In [7]:
import sys

In [8]:
# Consigue todas las publicaciones desde la primer pagina hasta la numero pages inclusive y las devuelve como lista
def get_house_links(pages, casa=True):
    homes = []
    for i in range(1,pages+1):
        sys.stdout.write(f'\rScrapeando pagina {i}')
        homes += get_links(base_url, page=i, casa=casa)
    
    return homes

In [9]:
house_links = get_house_links(360)

Scrapeando pagina 360

In [10]:
apartment_links = get_house_links(360, casa=False)

Scrapeando pagina 360

In [11]:
# Scrapea todas las casas de la lista conseguida por get_house_links
def scrape_homes(homes_list):
    homes = []
    count = 0
    for home in homes_list:
        home_info = get_info_from_house(home)

        if home_info is None:
            continue

        homes.append(home_info)
        count += 1
        sys.stdout.write(f"\rElementos scrapeados: {count}/{len(homes_list)}")

    return homes

In [12]:
houses = scrape_homes(house_links)

Elementos scrapeados: 2250/4320No se puede acceder a https://rentahouse.com.ve/casa_en_venta_en_yaritagua_en_municipio-pena_rah-23-15488.html
Elementos scrapeados: 2309/4320No se puede acceder a https://rentahouse.com.ve/casa_en_venta_en_barquisimeto_en_colinas-de-santa-rosa_rah-23-14955.html
Elementos scrapeados: 2590/4320No se puede acceder a https://rentahouse.com.ve/casa_en_venta_en_caracas_en_colinas-de-bello-monte_rah-23-13072.html
Elementos scrapeados: 3567/4320No se puede acceder a https://rentahouse.com.ve/casa_en_venta_en_municipio-vargas_en_caruao_rah-23-5789.html
Elementos scrapeados: 4316/4320

In [13]:
apartments = scrape_homes(apartment_links)

Elementos scrapeados: 4320/4320

Siendo sincero, la verdad no me gusta como scrapee las publicaciones. Esto es porque lo hice de manera secuencial, lo cual toma mucho tiempo.  
Hubiera preferido scrapear de forma paralela, por ejemplo con crontab + [click](https://click.palletsprojects.com/), o con [schedule](https://github.com/dbader/schedule), o incluso arreglármelas con asyncio u otra forma, pero decidí ir por lo simple (y feo) para asegurarme que funcione.

# Limpieza y transformaciones con pandas

Ahora queda limpiar la información de cada casa, para esto cargo los datos obtenidos en dataframes de pandas.

In [14]:
import pandas as pd
import numpy as np

In [15]:
houses_df = pd.DataFrame(houses)

In [16]:
houses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4316 entries, 0 to 4315
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Price                        4316 non-null   object
 1   Codígo RAH:                  4316 non-null   object
 2   Tipo de Propiedad:           4316 non-null   object
 3   Estilo:                      4316 non-null   object
 4   Área Privada:                4316 non-null   object
 5   Terreno:                     4316 non-null   object
 6   Estado Del Inmueble:         4316 non-null   object
 7   Dormitorios:                 4316 non-null   object
 8   Total Baños:                 4316 non-null   object
 9   Baños Completos:             4316 non-null   object
 10  Medios Baños:                1643 non-null   object
 11  Tipo De Estacionamiento:     4316 non-null   object
 12  Puestos De Estacionamiento:  3778 non-null   object
 13  Amoblado:                    4316

De la celda de arriba se puede observar que todas las columnas son strings, y que tienen nombres "feos". Para solucionar esto primero voy a renombrar las columnas y luego revisar sus tipos.

In [17]:
def rename_columns(df):
    new_col_names = {}
    for col in df.columns:
        old_col = str(col)
        if old_col.endswith(':') or old_col.endswith(': '):
            new_col = old_col.split(':')[0]
            new_col_names[old_col] = new_col

    df.rename(columns=new_col_names, inplace=True)
    df.rename(columns = {'✅Dormitorio De Servicio' : 'Dormitorio De Servicio'}, inplace=True)


In [18]:
rename_columns(houses_df)

In [19]:
apartments_df = pd.DataFrame(apartments)
rename_columns(apartments_df)
apartments_df.columns

Index(['Price', 'Codígo RAH', 'Tipo de Propiedad', 'Estilo', 'Área Privada',
       'Estado Del Inmueble', 'Dormitorios', 'Total Baños', 'Baños Completos',
       'Medios Baños', 'Tipo De Estacionamiento', 'Puestos De Estacionamiento',
       'Amoblado', 'Dormitorio De Servicio', 'País', 'Estado', 'Ciudad',
       'Urbanización'],
      dtype='object')

In [20]:
# Reviso por las dudas que de hecho todos los precios sean "$<numero> USD"
houses_df['Price'].apply(lambda x: not str(x).endswith('USD')).any()

False

In [21]:
def fix_prices(df):
    if not df['Price'].apply(lambda x: not str(x).endswith('USD')).any():
        df['Price'] = df['Price'].str.replace(r'\D+','', regex=True)
    df['Price'] = df['Price'].astype(int)
        

In [22]:
fix_prices(houses_df)
fix_prices(apartments_df)

In [23]:
houses_df.head()

,Price,Codígo RAH,Tipo de Propiedad,Estilo,Área Privada,Terreno,Estado Del Inmueble,Dormitorios,Total Baños,Baños Completos,Medios Baños,Tipo De Estacionamiento,Puestos De Estacionamiento,Amoblado,Dormitorio De Servicio,Calle,País,Estado,Ciudad,Urbanización
0,150000,VE 23-26807,Casa,Duplex,225 m2\n\n,225 m2\n\n,Usado,4,5,4,1,Cubierto,2,No,Si,Publica,Venezuela,Distrito Metropolitano,Caracas,La Boyera
1,50000,VE 23-26803,Casa,1 Nivel,178 m2\n\n,178 m2\n\n,Usado,6,5,5,NaN,Ninguno,NaN,No,NaN,Publica,Venezuela,Distrito Metropolitano,Caracas,La Pastora
2,150000,VE 23-26802,Casa,1 Nivel,630 m2\n\n,630 m2\n\n,Usado,5,5,4,1,Descubierto,2,Si,NaN,Cerrada con Vigilancia,Venezuela,Portuguesa,Araure,Araguaney
3,78000,VE 23-26799,Casa,Duplex,320 m2\n\n,200 m2\n\n,Usado,4,4,3,1,Cubierto,3,Parcialmente,Si,Cerrada con Vigilancia,Venezuela,Portuguesa,Araure,Maria Gabriela
4,120000,VE 23-26796,Casa,Duplex,212 m2\n\n,178 m2\n\n,Usado,3,3,2,1,Cubierto,3,Si,Si,Cerrada con Vigilancia,Venezuela,Portuguesa,Araure,Las Mesetas de Araure


In [24]:
def fix_numeric_columns(df):
    num_cols = ['Dormitorios', 'Total Baños', 'Baños Completos', 'Puestos De Estacionamiento', 'Medios Baños']
    for col in num_cols:
        df[col] = pd.to_numeric(df[col])

In [25]:
fix_numeric_columns(houses_df)
fix_numeric_columns(apartments_df)

In [26]:
print("Columnas en comun: ", list(set(houses_df.columns) & set(apartments_df.columns)))
print("Columnas en casas pero no depas: ",list(set(houses_df.columns) - set(apartments_df.columns)))

Columnas en comun:  ['Price', 'Total Baños', 'Estado', 'Baños Completos', 'Área Privada', 'Medios Baños', 'Tipo de Propiedad', 'Tipo De Estacionamiento', 'Ciudad', 'País', 'Estado Del Inmueble', 'Dormitorios', 'Codígo RAH', 'Urbanización', 'Estilo', 'Puestos De Estacionamiento', 'Dormitorio De Servicio', 'Amoblado']
Columnas en casas pero no depas:  ['Terreno', 'Calle']


In [27]:
def fix_areas(df, cols):
    for col in cols:
        df[col] = df[col].str.replace(' m2\n\n', '')
        df[col] = pd.to_numeric(df[col])

In [28]:
fix_areas(houses_df, ['Terreno', 'Área Privada'])
fix_areas(apartments_df, ['Área Privada'])

In [29]:
houses_df['Medios Baños'].value_counts(dropna=False)

NaN     2673
1.0     1134
2.0      243
0.0      170
3.0       70
4.0       10
6.0        7
5.0        4
11.0       1
7.0        1
10.0       1
8.0        1
9.0        1
Name: Medios Baños, dtype: int64

In [30]:
# Asumo que un medio baño nulo significa que tiene 0 medios baños
houses_df['Medios Baños'] = houses_df['Medios Baños'].fillna(0)
apartments_df['Medios Baños'] = apartments_df['Medios Baños'].fillna(0)

In [31]:
houses_df['Dormitorio De Servicio'].value_counts(dropna=False)

NaN    2610
Si     1706
Name: Dormitorio De Servicio, dtype: int64

In [32]:
# Reemplazo nulos por no
houses_df['Dormitorio De Servicio'] = houses_df['Dormitorio De Servicio'].fillna('No')
apartments_df['Dormitorio De Servicio'] = apartments_df['Dormitorio De Servicio'].fillna('No')

In [33]:
houses_df['Puestos De Estacionamiento'].value_counts(dropna=False)

2.0      1003
4.0       705
3.0       655
NaN       538
6.0       328
5.0       306
1.0       223
10.0      175
8.0       151
7.0        69
12.0       35
0.0        29
9.0        21
20.0       20
15.0       17
14.0        9
30.0        5
40.0        4
11.0        4
18.0        3
25.0        3
23.0        2
16.0        2
24.0        1
21.0        1
19.0        1
56.0        1
50.0        1
13.0        1
34.0        1
100.0       1
60.0        1
Name: Puestos De Estacionamiento, dtype: int64

In [34]:
houses_df[(houses_df['Tipo De Estacionamiento'] != 'Ninguno') & (houses_df['Puestos De Estacionamiento'].isnull())][['Tipo De Estacionamiento', 'Puestos De Estacionamiento']].head()

,Tipo De Estacionamiento,Puestos De Estacionamiento
10,Cubierto,NaN
20,Cubierto,NaN
21,Cubierto,NaN
56,Cubierto,NaN
62,Cubierto,NaN


Inicialmente, creí que un puesto de estacionamiento nulo quería decir que no había puestos de estacionamiento, pero esto es incorrecto ya que hay publicaciones con tipo de estacionamiento distinto de "ninguno" pero con puestos de estacionamiento nulo. Debido a esto, no reemplace los nulos y los deje como están.

# Exportación a CSV

Ahora con los dataframes limpiados, los exporto a csv para persistirlos.

In [35]:
houses_df.rename(columns={'Price':'Precio'}, inplace=True)
apartments_df.rename(columns={'Price':'Precio'}, inplace=True)

In [36]:
houses_df.to_csv('Casas_venezuela.csv', index=False)
apartments_df.to_csv('Departamentos_venezuela.csv', index=False)